# "iso 1861 UTC" "lon" "lat"  transfer to local time

## Method 1. Bing Map API

use bing map API https://docs.microsoft.com/en-us/bingmaps/rest-services/timezone/convert-local-time-zone

In [67]:
import datetime
import json
import os
import requests
import pandas as pd

In [71]:
# Using Bing Maps API
BING_MAPS_KEY ="AhsT3Omaszmnue8vawflFt6FuwnJYkrhxy9EwGNKqp8wt0jn_pqTkY3qpdcZf54S"
print(BING_MAPS_KEY )
#AtpaOr6ss6CaEeNNDrVMBINBfospfvbMiF7FhhnyGgL5y0fIM9QvSZZc0n5er1Xu

AhsT3Omaszmnue8vawflFt6FuwnJYkrhxy9EwGNKqp8wt0jn_pqTkY3qpdcZf54S


In [69]:
#working
def get_time_zone(lon,lat,isotime_utc):
    try: 
        r = requests.get(
        f"https://dev.virtualearth.net/REST/v1/TimeZone/{lat},{lon}?datetime={isotime_utc}&key={BING_MAPS_KEY}")
    
        return r.json()['resourceSets'][0]['resources'][0]['timeZone']['convertedTime']['localTime']
    except:
        return "NA"

In [76]:
print(get_time_zone(-78.853563,42.8962295,"2019-03-31T23:57:12.000Z"))

2019-03-31T19:57:12


In [74]:
#csv add local_time column
import pandas as pd
df= pd.read_csv(r"E:\OneDrive - University of Georgia\Project\Data\tweet_data_3_groccery\4.1 LocalTime\groceryStore_bboxStateLevel_localTime2019_2.csv",low_memory=False)

In [75]:
pd.options.mode.chained_assignment = None
i=0
#219307
for row in df.head(225000).iterrows():
    if  str(df['local_time'][i])[0]!="2":
        df['local_time'][i]= get_time_zone(df["lon"][i],df["lat"][i],df["created_at"][i])  
        print(i,df["created_at"][i],df['local_time'][i])
    #print(i)
    i+=1

4166 2020-01-12T21:16:47.000Z NA
13338 2019-12-25T16:11:42.000Z NA
29247 2019-11-30T19:34:38.000Z NA
29622 2019-11-30T03:47:31.000Z NA
29927 2019-11-29T21:25:06.000Z NA
36125 2019-11-20T22:45:25.000Z NA
51234 2019-10-25T20:45:49.000Z NA
55585 2019-10-17T02:51:13.000Z NA
69828 2019-09-21T19:14:05.000Z NA
71093 2019-09-19T21:01:59.000Z NA
72238 2019-09-17T20:30:51.000Z NA
88074 2019-08-27T00:38:18.000Z NA
88075 2019-08-27T00:37:54.000Z NA
93388 2019-08-18T18:44:11.000Z NA
96684 2019-08-14T01:04:12.000Z NA
102048 2019-08-09T01:45:03.000Z NA
105353 2019-08-05T15:31:40.000Z NA
116416 2019-07-21T21:42:04.000Z NA
123802 2019-07-09T13:08:12.000Z NA
132895 2019-06-24T00:37:44.000Z NA
142079 2019-06-08T19:16:20.000Z NA
143203 2019-06-06T23:57:36.000Z NA
146217 2019-06-02T02:03:58.000Z NA
153294 2019-05-19T22:36:44.000Z NA
153986 2019-05-18T18:17:42.000Z NA
165896 2019-04-25T16:59:19.000Z NA
166390 2019-04-24T18:48:01.000Z NA
167935 2019-04-21T17:50:02.000Z NA
175583 2019-04-07T19:18:52.000Z NA
1

In [66]:
df.to_csv(r"E:\OneDrive - University of Georgia\Project\Data\tweet_data_3_groccery\4.1 LocalTime\groceryStore_bboxStateLevel_localTime2019_2.csv",encoding="utf_8_sig",index=False)

## Method2. Use datetime and pytz library

In [ ]:
from tzwhere import tzwhere
from pytz import timezone
import datetime
import pytz
import time
from dateutil import parser

In [ ]:
#iso to timestamp
import datetime,pytz

def iso2timestamp(datestring, format='%Y-%m-%dT%H:%M:%S.%fZ',timespec='seconds'):
    """
    ISO8601时间转换为时间戳

    :param datestring:iso时间字符串 2019-03-25T16:00:00.000Z，2019-03-25T16:00:00.000111Z
    :param format:%Y-%m-%dT%H:%M:%S.%fZ；其中%f 表示毫秒或者微秒
    :param timespec:返回时间戳最小单位 seconds 秒，milliseconds 毫秒,microseconds 微秒
    :return:时间戳 默认单位秒
    """
    tz = pytz.timezone('Asia/Shanghai')
    utc_time = datetime.datetime.strptime(datestring, format)  # 将字符串读取为 时间 class datetime.datetime

    time = utc_time.replace(tzinfo=pytz.utc).astimezone(tz)

    times = {
        'seconds': int(time.timestamp()),
        'milliseconds': round(time.timestamp() * 1000),
        'microseconds': round(time.timestamp() * 1000 * 1000),
    }
    return times[timespec]

In [ ]:
def get_local_time(lon,lat,isotime_utc):
    
    try: 
        #iso to timestamp
        timestamp = iso2timestamp(isotime_utc)
    
        #timestamp, lat, lon to local_time
        tz = tzwhere.tzwhere(forceTZ=True)
        time_zone = tz.tzNameAt(lat,lon, forceTZ=True)  #纬度，经度。 得到对应的时区
        #print(timestamp)
        local_time = datetime.datetime.fromtimestamp(timestamp,pytz.timezone(time_zone)).strftime('%Y-%m-%dT%H:%M:%S')
    
        return str(local_time)
    
    except:
        return "NA"

In [ ]:
print(get_local_time(-78.853563,42.8962295,"2019-03-31T23:57:12.000Z"))

In [ ]:
#csv add local_time column
import pandas as pd
df= pd.read_csv(r"Estate_level.csv",low_memory=False) 

In [ ]:
#TRY THIS
#Recalculate the NA timezone
pd.options.mode.chained_assignment = None
i=0

for row in df.iterrows():
    if  str(df['local_time'][i])=="nan":
        df['local_time'][i]= get_local_time(df["lon"][i],df["lat"][i],df["created_at"][i])  
        print(i)
        print(df['local_time'][i])
   
    i+=1

In [ ]:
df.to_csv(r"E:\Project\Data\tweet_data\3.1 FilterBBox4Level_4csv\2.csv",encoding="utf_8_sig",index=False)